In [1]:
%load_ext lemma.ipython

In [2]:
%%hy

(require [lemma.core :as le])
(import [lemma.exceptions [LeEquationError]])
(import [lemma.domain.algebra [seq-sum pow length add sub PI
                               div/frac :as div
                               mul/times :as mul]])

In [3]:
print('foo')
%hy (print "bar")
%lemma (add 1 2)

foo
bar


$$1 + 2$$,3


In [4]:
%%lemma

PI
(div 22 7)
(add 3 0.14)

$$\pi$$,3.141592653589793
$$\frac{22}{7}$$,3.142857142857143
$$3 + 0.14$$,3.14


## Simple expression

In [5]:
%%hy

;; Compile a lemma expression with le.expr
(setv simple-expression
 (le.expr (add 1 PI)))

simple-expression

$$1 + \pi$$,4.141592653589793


In [6]:
%%hy

;; Generate latex for a lemma expression 
(.latex simple-expression)

'1 + \\pi'

In [7]:
%%hy

;; Execute a lemma expression like a function
(simple-expression)

4.141592653589793

## More complex example (with `seq-sum` and an identifier)

In [8]:
%%hy

;; Specify latex formatter for variable names.
(le.def-identifier beta r"\beta")

(setv complex-expression
 (le.expr
  (seq-sum [a [1 2]
            beta [3 4]]
   (sub 1 (div a beta)))))

(display (.latex complex-expression))
(print (complex-expression))

'\\sum_{a \\in \\{1, 2\\}, \\beta \\in \\{3, 4\\}} 1 - \\frac{a}{\\beta}'

2.25


In [9]:
%%hy

(import [hy.contrib.hy-repr [hy-repr]])

;; You can even print the Hy code for an expression.
(print (hy-repr (.hy complex-expression)))

'(sum (lfor a [1 2] beta [3 4] (do (require hy.contrib.walk) (hy.contrib.walk.let [args [1 (do (require hy.contrib.walk) (hy.contrib.walk.let [numerator a denominator beta] (do (/ numerator denominator))))]] (do (hy.core.shadow.- #* args))))))


## Order of operations

In [10]:
%%hy

;; Parens will be automatically added based on operator precedence.
(setv order-examples
 [(le.expr (sub (add 1 2) 3))
  (le.expr (sub 1 (add 2 3)))
  (le.expr (sub 1 (div 2 3)))])

(for [example order-examples]
  (display (.latex example))
  (print (example))
  (print "-----"))

'\\left(1 + 2\\right) - 3'

0
-----


'1 - \\left(2 + 3\\right)'

-4
-----


'1 - \\frac{2}{3}'

0.33333333333333337
-----


In [11]:
%%hy

;; Use #p to explicitly add parens.
(setv parens-example (le.expr (sub 1 #p(div 2 3))))
(display (.latex parens-example))
(print (parens-example))

'1 - \\left(\\frac{2}{3}\\right)'

0.33333333333333337


In [12]:
%%hy

;; Use #b to explicitly remove parens.
(setv bare-example (le.expr (add 1 #b(add 2 3))))
(display (.latex bare-example))
(print (bare-example))

'1 + 2 + 3'

6


## Formulas

In [13]:
%%hy

(le.def-identifier xs "X")
(le.def-identifier mu r"\mu")

(le.def-formula variance r"\sigma^2"
  [xs mu n]
  "Formula for population variance."
  (div
   (seq-sum [x xs]
    (pow (sub x mu) 2))
   n))

In [14]:
%%hy

;; Use a formula like any other operator
(setv var-expression (le.expr (variance [1 2 3] :mu 2 :n 3)))
(display (.latex var-expression))
(print (var-expression))

'\\sigma^2\\left(\\{1, 2, 3\\}, 2, 3\\right)'

0.6666666666666666


In [15]:
%%hy

;; Or use a formula as an operator whose latex representation is the formula's body
(setv var-expression (le.expr (variance.op [1 2 3] 2 3)))
(display (.latex var-expression))
(print (var-expression))

'\\frac{\\sum_{x \\in \\{1, 2, 3\\}} \\left(x - 2\\right)^{2}}{3}'

0.6666666666666666


In [16]:
%%hy

;; Get the signature of a formula
(.signature-latex variance)

'\\sigma^2\\left(X, \\mu, n\\right)'

In [17]:
%%hy

;; Get the definition of a formula
(display (.latex variance))
(display (.latex variance [1 2 3] 2 3))

'\\sigma^2\\left(X, \\mu, n\\right) = \\frac{\\sum_{x \\in X} \\left(x - \\mu\\right)^{2}}{n}'

'\\sigma^2\\left(\\{1, 2, 3\\}, 2, 3\\right) = \\frac{\\sum_{x \\in \\{1, 2, 3\\}} \\left(x - 2\\right)^{2}}{3}'

In [18]:
%%hy

;; Directly run a formula like a function
(variance :xs [0 5 10] :mu 5 :n 3)

16.666666666666668

In [19]:
%%hy

;; Get docstring.
variance.--doc--

'Formula for population variance.'

## Python Interop

In [20]:
# Use expressions and formulae directly from Python:

display(variance.signature_latex())
display(variance.latex())
print(variance(xs=[0, 5, 10], mu=5, n=3))
display(var_expression.latex())
print(var_expression())

'\\sigma^2\\left(X, \\mu, n\\right)'

'\\sigma^2\\left(X, \\mu, n\\right) = \\frac{\\sum_{x \\in X} \\left(x - \\mu\\right)^{2}}{n}'

16.666666666666668


'\\frac{\\sum_{x \\in \\{1, 2, 3\\}} \\left(x - 2\\right)^{2}}{3}'

0.6666666666666666


## Equations

In [21]:
%%hy

(le.def-identifier z r"\zeta")

;; Equations are defined as a series of equivalent expressions.
;; Useful for working out equation solutions with executable code and then printing out notation.
(le.def-equation my-equation
                 [x &optional [z 4]]
                 "Equation involving FOIL expansion."
                 [(add #b(mul (add x 1) (sub x 1)) z)
                  (add (sub #b(add #b(sub (pow x 2) x) x) 1) z)
                  (add (sub (pow x 2) 1) z)])

In [22]:
%%hy

(display (.latex my-equation))
(display (.latex my-equation :x 5))
(my-equation :x 5)

'\\begin{aligned} \\left(x + 1\\right) \\times \\left(x - 1\\right) + \\zeta &= \\left(x^{2} - x + x - 1\\right) + \\zeta \\\\&= \\left(x^{2} - 1\\right) + \\zeta\\end{aligned}'

'\\begin{aligned} \\left(5 + 1\\right) \\times \\left(5 - 1\\right) + 4 &= \\left(5^{2} - 5 + 5 - 1\\right) + 4 \\\\&= \\left(5^{2} - 1\\right) + 4\\end{aligned}'

28

In [23]:
%%hy

(setv eq-expr (le.expr (add 1 (my-equation 5))))
(display (.latex eq-expr))
(print (eq-expr))

'1 + \\left(\\left(5 + 1\\right) \\times \\left(5 - 1\\right) + 4\\right)'

29


In [24]:
%%hy

my-equation.--doc--

'Equation involving FOIL expansion.'

In [25]:
%%hy

(le.def-equation bad-equation [x]
                 [(add x 1)
                  (sub x 1)])

(display (.latex bad-equation))

;; When executing an equation, all expressions are executed,
;; and an exception is raised if any of the results were not equal.
(try
 (bad-equation :x 5)
 (except [ex LeEquationError]
  (print ex)))

'\\begin{aligned} x + 1 &= x - 1\\end{aligned}'

While evaluating LeEquation#bad-equation with arguments [x=5]: result '4' of LeExpression#(LeCallableOperator#sub 5 1) did not equal result '6' of LeExpression#(LeCallableOperator#sub 5 1)


## Unit Testing

In [26]:
%%hy

(assert (= ((le.expr (add 1 1)) 2)))
(assert (= (variance [1 2 3] 2 3) (/ 2 3)))

In [27]:
assert var_expression() == 2 / 3
assert variance([1, 2, 3], 2, 3) == 2 / 3

## Extending Lemma

In [28]:
%%hy

;; Define your own constants.
(import math)
(le.def-constant e "e" math.e)

(display (.latex e))
(print (e))

'e'

2.718281828459045


In [29]:
%%hy

;; Define operators using lemma expressions.
(le.def-operator decrement [val]
  (expr (sub val 1)))

(setv dec-expr (le.expr (decrement 3)))
(display (.latex dec-expr))
(print (dec-expr))

'3 - 1'

2


In [30]:
%%hy

;; Define operators using Hy expressions.
(le.def-operator increment [val]
  (precedence 100)
  (latex f"{val} + 1")
  (hy (+ val 1)))

(setv inc-expr (le.expr (increment 3)))
(display (.latex inc-expr))
(print (inc-expr))

'3 + 1'

4


In [31]:
%%hy

(import [lemma.lang [gen-latex gen-hy]])

;; Define operators using Hy macros (arguments are pass-by-name, and
;; hy-macro is expected to return a quoted Hy expression). Useful for
;; more complex operators that don't just take lemma expressions as
;; arguments (like seq-sum).
(le.def-operator plus2 [val]
  (precedence 100)
  (latex-macro
   f"{(gen-latex val)} + 2")
  (hy-macro
   `(+ ~(gen-hy val) 2)))

(setv plus2-expr (le.expr (plus2 3)))
(display (.latex plus2-expr))
(print (plus2-expr))

'3 + 2'

5


In [32]:
%%hy

;; Functions can accept optional args with default values (&rest and &kwargs also work).
(le.def-operator mean
  [vals &optional [val x]]
  (expr (seq-sum [val vals] (div val (length vals)))))

(le.def-identifier g r"\gamma")

(setv mean-example (le.expr (add (mean [1 2 3]) (mean [4 5 6] y) (mean [7 8 9] g))))
(display (.latex mean-example))
(print (mean-example))

'\\left(\\sum_{x \\in \\{1, 2, 3\\}} \\frac{x}{|\\{1, 2, 3\\}|}\\right) + \\left(\\sum_{y \\in \\{4, 5, 6\\}} \\frac{y}{|\\{4, 5, 6\\}|}\\right) + \\left(\\sum_{\\gamma \\in \\{7, 8, 9\\}} \\frac{\\gamma}{|\\{7, 8, 9\\}|}\\right)'

15.0


## Next Steps

* [API reference](https://ben-denham.github.io/lemma/#/lemma.core)
* [Source-code for `lemma.domain.algebra`](https://github.com/ben-denham/lemma/blob/master/lemma/domain/algebra.hy) (useful reference for writing your own domains of operators)
* [Documentation home](https://ben-denham.github.io/lemma)